# Chap05 - 텍스트 1: 텍스트와 시퀀스 처리 및 텐서보드 시각화

> 텐서플로에서 시퀀스(sequence) 데이터인 텍스트를 어떻게 다루는지 알아보고, RNN 구현방법 및 텐서보드를 이용한 시각화에 대해 알아본다. 그 다음 단어 임베딩 학습 및 LSTM을 구현해본다.

## 5.1 시퀀스 데이터의 중요성

[Chap04-합성곱 신경망 CNN](http://excelsior-cjh.tistory.com/152)에서 이미지의 공간(spatial) 구조를 이용하여 CNN을 구현하였고, 이러한 구조를 활용하는 것이 중요하다는 것을 알아 보았다. 이번에 알아볼 순차형 데이터 구조인 시퀀스(sequence) 데이터 또한 중요하고 유용한 구조이다. 시퀀스 데이터란 각각의 데이터가 순서가 있는 데이터를 말하며, 다양한 분야에서 찾을 수가 있다. 예를 들어, 음성신호, 텍스트, 주가 데이터 등이 있다.

<img src="./images/sequence_data.png" width="60%" height="60%"/>

## 5.2 RNN 소개

**RNN(순환신경망, Recurrent Neural Network)**은 시퀀스 데이터의 모델링에 사용되는 신경망 구조이다. RNN 모델의 바탕에는 시퀀스에서 현재 이후의 각 데이터는 새로운 정보를 제공하므로, 이 정보로 모델의 현재 상태를 **'갱신(업데이트)'** 한다는 아이디어가 깔려있다.

어떤 텍스트에서 문장을 읽을 때 각각의 새로운 단어로 현재 상태의 정보가 갱신되는데 이 상태는 새롭게 나타난 단어뿐만 아니라 이전의 단어에 대해서도 종속적이다.

머신러닝에서 시퀀스 패턴의 데이터를 모델링하기 위해 흔히 사용되는 통계 및 확률기반의 [마르코프 체인](https://en.wikipedia.org/wiki/Markov_chain)(Markov Chain) 모델이다. 데이터를 시퀀스를 '체인'으로 본다면, 체인의 각 노드는 이전 노드로부터 어떤 식으로든 종속적이므로 '과거'는 지워지지 않고 이어진다.

RNN 모델 또한 체인 구조 개념을 기반으로 하고 있으며 정보를 유지하고 갱신하는 방법에 따라 다양한 종류가 있다. '순환'이라는 이름에서 알 수 있듯이 RNN은 일종의 **'루프'**로 이루어진다.



![](./images/rnn.png)

위의 그림에서 볼 수 있듯이, 시점 $t$에서 네트워크는 입력값 $x_t$ (문장 중 하나의 단어)를 관찰하고, '상태 벡터'(state vector, $t$ 시점의 출력, $h_t$)를 이전의 $h_{t-1}$에서 $h_t$로 업데이트 한다. 새로운 입력(다음 단어)은 $t-1, t-2, \dots$에서 관찰한 이전 입력이 현재($t$) 입력의 이해에 영향을 미치므로 과거 시퀀스에 종속적이다. 위의 그림에서 처럼, 이러한 순환구조를 길게 펼쳐놓은 체인으로 생각할 수 있다. 

### 5.2.1 기본적인 RNN 구현

이제 텐서플로(TensorFlow)를 사용해 시퀀스 데이터를 다루는 방법과 기초적인 RNN을 구현해보도록 하자.

먼저, RNN 모델의 업데이트 단계에 대해 알아보면 RNN의 업데이트 단계는 다음과 같이 나타낼 수 있다.

<img src="./images/rnn02.png" width="70%" height="70%"/>

이를 수식으로 나타내면 아래와 같다.

$$
h_t = \tanh{\left( \mathrm{W}_x x_t + \mathrm{W}_h h_{t-1} + b \right)}
$$

$\mathrm{W}_x, \mathrm{W}_h, b$ 는 학습할 가중치(weight) 및 편향값(bias)의 변수이며, 활성화 함수로는 하이퍼볼릭 탄젠트 함수($\tanh$)를 사용했다. $x_t$와 $h_t$는 입력과 상태 벡터이다.

#### 시퀀스로서의 MNIST 이미지